In [10]:
PRFX='0316_4'
PRFX_PRP = 'prep0316_1'
trntmstmp=1584412344
valtmstmp=1584405047
import datetime
[datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (trntmstmp, valtmstmp)]

['2020-03-17 02:32:24', '2020-03-17 00:30:47']

# setup

In [22]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_curve, auc
import pickle
import lightgbm as lgb

def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

SEED=101
HOME='/data/git/recsys20'
p_in=f'{HOME}/output/{PRFX_PRP}'
p_out=f'{HOME}/output/{PRFX}'
Path(p_out).mkdir(exist_ok=True, parents=True)

from sklearn.metrics import precision_recall_curve, auc, log_loss

def compute_prauc(pred, gt):
    prec, recall, thresh = precision_recall_curve(gt, pred)
    prauc = auc(recall, prec)
    return prauc

def calculate_ctr(gt):
    positive = len([x for x in gt if x == 1])
    ctr = positive/float(len(gt))
    return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

In [29]:
tgts=['Retweet','Reply','Like','RTwCmnt',]
cols_tgt=['retweet', 'reply', 'like', 'retweet_with_comment']

In [32]:
dftrn = pd.read_csv(f'{p_in}/dftrn_prep_{trntmstmp}__{PRFX_PRP}.csv', nrows=5)
dfval = pd.read_csv(f'{p_in}/dfval_prep_{valtmstmp}__{PRFX_PRP}.csv', nrows=5)

In [33]:
datatrn=lgb.Dataset(f'{p_in}/dftrn_prep_{trntmstmp}__{PRFX_PRP}.csv')
dataval=lgb.Dataset(f'{p_in}/dfval_prep_{valtmstmp}__{PRFX_PRP}.csv')

In [34]:
dftrn.head()

,timestamp,engaged_follower_count,engaged_following_count,engaged_is_verified,engaged_account_creation_time,engaging_follower_count,engaging_following_count,engaging_is_verified,engaging_account_creation_time,engagee_follows_engager,...,tmdelta_engng_enged,tmdelta_enged_tweet,tmdelta_engng_tweet,tweet_type_TopLevel,tweet_type_Retweet,tweet_type_Quote,retweet,reply,like,retweet_with_comment
0,1581258144,3517,590,False,1391702737,27428,600,False,1520948869,False,...,129246132,-189555407,-60309275,1,0,0,0,0,0,0
1,1581018910,39786,32978,False,1322740272,17,77,False,1569692352,True,...,246952080,-258278638,-11326558,1,0,0,0,0,0,0
2,1581244866,141726,3232,False,1343181073,2,29,False,1568107028,False,...,224925955,-238063793,-13137838,1,0,0,0,0,1,0
3,1581250124,3044,99,False,1350921963,3,30,False,1568107028,False,...,217185065,-230328161,-13143096,1,0,0,0,0,0,0
4,1580958805,141370,3235,False,1343181073,3,30,False,1568107028,False,...,224925955,-237777732,-12851777,1,0,0,0,0,0,0


In [38]:
cols_feat, cols_tgt=dftrn.columns[:-4], dftrn.columns[-4:]

In [46]:
ds=lgb.Dataset(data=f'{p_in}/dftrn_prep_{trntmstmp}__{PRFX_PRP}.csv',
                    label=cols_tgt, feature_name=cols_feat)

In [47]:
dsval=lgb.Dataset(data=f'{p_in}/dfval_prep_{valtmstmp}__{PRFX_PRP}.csv', feature_name=cols_feat)

In [48]:
param = {'num_leaves': 31, 'objective': 'binary', 'metric':['auc', 'binary_logloss']}

In [49]:
param

{'num_leaves': 31, 'objective': 'binary', 'metric': ['auc', 'binary_logloss']}

In [50]:
bst = lgb.train(param, ds, num_boost_round=10, valid_sets=[dsval])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()